In [1]:
import pretty_midi as midi

In [26]:
# Configurations
FILE_PATH = './files/midi/piano.mid'
OUT_FILE_PATH = './files/midi/piano-separated.mid'
DEFAULT_INSTRUMENT = 'Acoustic Grand Piano'

# Read MIDi file and clean up
score = midi.PrettyMIDI(FILE_PATH)
score.remove_invalid_notes()
print('Loaded "{}".'.format(FILE_PATH))

Loaded "./files/midi/piano.mid".


In [27]:
# Group all notes by start time
note_by_tick = {}
note_counter = 0
max_group_size = 0

for instrument in score.instruments:
    for note in instrument.notes:
        tick = score.time_to_tick(note.start)
        if not tick in note_by_tick:
            note_by_tick[tick] = []
        note_by_tick[tick].append(note)
        max_group_size = max(max_group_size, len(note_by_tick[tick]))
        note_counter += 1
        
print('Found {} distinguishable start times for {} notes. '
      'Largest chord has {} notes.'.format(len(note_by_tick),
                                           note_counter,
                                           max_group_size))

Found 2800 distinguishable start times for 4767 notes. Largest chord has 7 notes.


In [28]:
# Create a new MIDI file
new_score = midi.PrettyMIDI()

# Copy data from old score
new_score.time_signature_changes = score.time_signature_changes
new_score.key_signature_changes = score.key_signature_changes

# Create as many parts as we need to keep all voices separate
for instrument_index in range(0, max_group_size):
    program = midi.instrument_name_to_program(DEFAULT_INSTRUMENT)
    new_instrument = midi.Instrument(program=program)
    new_score.instruments.append(new_instrument)
    
# Assign notes to different parts
for notes in note_by_tick.values():
    for instrument_index, note in enumerate(notes):
        new_score.instruments[instrument_index].notes.append(note)
        
for index, instrument in enumerate(new_score.instruments):
    print('{} notes in part {}.'.format(len(instrument.notes), index + 1))

2800 notes in part 1.
1169 notes in part 2.
504 notes in part 3.
213 notes in part 4.
73 notes in part 5.
7 notes in part 6.
1 notes in part 7.


In [29]:
# Save result
new_score.write(OUT_FILE_PATH)